# Movie Analysis Project by Brooke Smyth, Garrett Williams, and Scott Schumann

**Overview:** [LET'S FILL IN A ONE PARAGRAPH OVERVIEW OF THE PROJECT LATER]

**Business Problem:** [SUMMARY OF BUSINESS PROBLEM TO BE FILLED IN LATER]

**Data Understanding:** [DESCRIPTION OF DATA BEING USED IN THIS PROJECT TO BE FILLED IN LATER]

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

**Data Preparation:** [DESCRIBE AND JUSTIFY THE PROCESS OF PREPARING DATA FOR ANALYSIS TO BE FILLED IN LATER]

In [2]:
imdb_name = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
imdb_title = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
imdb_crew = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
imdb_principals = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
rt_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter="\t")
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep="\t", encoding='latin-1')
tn = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

In [3]:
# Merging dataframes
tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [4]:
#create a new column that has the year as an integer
tn['year_int'] = tn['release_date'].str[-4:].astype(int)

#get info from movies since 2010 because that's as far back as the imdb data goes
tn = tn[tn['year_int'] >= 2015]

#convert the values in the worldwide gross column to integers
tn["ww_gross_int"] = tn["worldwide_gross"].replace("[\$,]", "", regex=True).astype(int)

#convert the values in the production budget column to integers
tn["pro_budget_int"] = tn["production_budget"].replace("[\$,]", "", regex=True).astype(int)

#create a new column that calculates return on investment 
tn["roi"] = (tn["ww_gross_int"] - tn["pro_budget_int"])/tn["pro_budget_int"]

#sort the movies by return on investment
tn.sort_values(by = "roi", ascending = False, inplace = True)
tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year_int,ww_gross_int,pro_budget_int,roi
5679,80,"Jul 10, 2015",The Gallows,"$100,000","$22,764,410","$41,656,474",2015,41656474,100000,415.564740
5213,14,"Apr 17, 2015",Unfriended,"$1,000,000","$32,789,645","$64,364,198",2015,64364198,1000000,63.364198
4249,50,"Jan 20, 2017",Split,"$5,000,000","$138,141,585","$278,964,806",2017,278964806,5000000,54.792961
4248,49,"Feb 24, 2017",Get Out,"$5,000,000","$176,040,665","$255,367,951",2017,255367951,5000000,50.073590
5063,64,"Oct 21, 2016",Moonlight,"$1,500,000","$27,854,931","$65,245,512",2016,65245512,1500000,42.497008
...,...,...,...,...,...,...,...,...,...,...
4995,96,"Mar 20, 2015",The Walking Deceased,"$2,000,000",$0,$0,2015,0,2000000,-1.000000
4997,98,"Aug 21, 2015",The Curse of Downers Grove,"$2,000,000",$0,$0,2015,0,2000000,-1.000000
4998,99,"Dec 31, 2015",Wind Walkers,"$2,000,000",$0,$0,2015,0,2000000,-1.000000
4999,100,"Jul 8, 2016",The Dog Lover,"$2,000,000",$0,$0,2016,0,2000000,-1.000000


In [5]:
imdb_title_rb = pd.merge(imdb_title, imdb_ratings, on = "tconst", how = "left")

In [6]:
imdb_title_crb = pd.merge(imdb_title_rb, imdb_crew, on = "tconst", how = "left")
imdb_title_crb.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,directors,writers
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,nm0002411,NaN
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,nm0000080,"nm0000080,nm0462648"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,nm0611531,nm0347899
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"nm0765384,nm0749914","nm1360635,nm0749914"


In [7]:
#imdb_merged = pd.merge(imdb_title_crb, imdb_name, left_on = "directors", right_on = "nconst", how = "left")
#imdb_merged.head()

In [8]:
#Merging the TN and IMDB dataframes, which only includes data from 2010 to the present. We feel this is 
#relevant because the stakeholder is trying to start a new studio and thus more recent data is probably
#more useful.
tn_imdb_merged = pd.merge(tn, imdb_title_crb, left_on = ['movie', 'year_int'], right_on = ['primary_title', 'start_year'], how = 'inner')
tn_imdb_merged['year_int'].value_counts()

2015    203
2016    164
2017    124
2018    120
2019     41
2020      1
Name: year_int, dtype: int64

**Data Modeling:**